In [1]:
# Imports
from pathlib import Path

import pandas as pd
import torch
from neuralhydrology.nh_run import start_run, eval_run, finetune

In [24]:
# Change these parameters:
pretrained_dir = "us_rep_ea"
pretrained_epoch = "020"
finetune_yaml = "finetune.yml"

# by default we assume that you have at least one CUDA-capable NVIDIA GPU
# if torch.cuda.is_available():
    
#     # start_run(config_file=Path("kz_basins.yml"))
#     start_run(config_file=Path("./conf/kz_basins.yml"))

# # fall back to CPU-only mode
# else:
#     start_run(config_file=Path("./conf/kz_basins.yml"), gpu=-1)

In [25]:
# Load validation results from the last epoch
import os

run_dir = Path(f"./runs/{pretrained_dir}")
# df = pd.read_csv(run_dir / "validation" / f"model_epoch{pretrained_epoch}" / "validation_metrics.csv", dtype={'basin': str})
# df = df.set_index('basin')

# Compute the median NSE from all basins, where discharge observations are available for that period
# print(f"Median NSE of the validation period {df['NSE'].median():.3f}")
# print(f"Mean NSE of the validation period {df['NSE'].mean():.3f}")
# print(df)
# Select a random basins from the lower 50% of the NSE distribution
# for i, row in df.iterrows():
#     print(row)
#     basin = row
#     print(f"Selected basin: {basin['Name']} with an NSE of {basin['NSE']:.3f}")

## Finetuning

Next, we will show how to perform finetuning for the basin selected above, based on the model we just trained. The function to use is `finetune` from `neuralhydrology.nh_run` if you want to train from within a script or notebook. If you want to start finetuning from the command line, you can also call the `nh-run` utility with the `finetune` argument, instead of e.g. `train` or `evaluate`.

The only thing required, similar to the model training itself, is a config file. This config however has slightly different requirements to a normal model config and works slightly different:
- The config has to contain the following two arguments:
    - `base_run_dir`: The path to the directory of the pre-trained model.
    - `finetune_modules`: Which parts of the pre-trained model you want to finetune. Check the documentation of each model class for a list of all possible parts. Often only parts, e.g. the output layer, are trained during finetuning and the rest is kept fixed. There is no general rule of thumb and most likely you will have to try both.
- Any additional argument contained in this config will overwrite the config argument of the pre-trained model. Everything _not_ specified will be taken from the pre-trained model. That is, you can e.g. specify a new basin file in the finetuning config (by `train_basin_file`) to finetune the pre-trained model on a different set of basins, or even just a single basin as we will do in this notebook. You can also change the learning rate, loss function, evaluation metrics and so on. The only thing you can not change are arguments that change the model architecture (e.g. `model`, `hidden_size` etc.), because this leads to errors when you try to load the pre-trained weights into the initialized model.

Let's have a look at the `finetune.yml` config that we prepared for this tutorial (you can find the file in the same directory as this notebook).

In [ ]:
!cat ./conf/{finetune_yaml}

So out of the two arguments that are required, `base_run_dir` is still missing. We will add the argument from here and point at the directory of the model we just trained. Furthermore, we point to a new file for training, validation and testing, called `finetune_basin.txt`, which does not yet exist. We will create this file and add the basin we selected above as the only basin we want to use here. The rest are some changes to the learning rate and the number of training epochs as well as a new name. Also note that here, we train the full model, by selecting all model parts available for the `CudaLSTM` under `finetune_modules`.

In [27]:
# Add the path to the pre-trained model to the finetune config
with open(f"./conf/{finetune_yaml}", "a") as fp:
    fp.write(f"\nbase_run_dir: {run_dir.absolute()}")
    
# Create a basin file with the basin we selected above
#with open("./basins/finetune_basin.txt", "w") as fp:
#    fp.write(basin)

With that, we are ready to start the finetuning. As mentioned above, we have two options to start finetuning:
1. Call the `finetune()` function from a different Python script or a Jupyter Notebook with the path to the config.
2. Start the finetuning from the command line by calling

```bash
nh-run finetune --config-file /path/to/config.yml
```

Here, we will use the first option.

In [ ]:
finetune(Path(f"./conf/{finetune_yaml}"))

Looking at the validation result, we can see an increase of roughly 0.05 NSE.

Last but not least, we will compare the pre-trained and the finetuned model on the test period. For this, we will make use of the `eval_run` function from `neuralhydrolgy.nh_run`. Alternatively, you could evaluate both runs from the command line by calling

```bash
nh-run evaluate --run-dir /path/to/run_directory/
```

In [ ]:
# eval_run(run_dir, period="test")

Now we can call the `eval_run()` function as above, but pointing to the directory of the finetuned run. By default, this function evaluates the last checkpoint, which can be changed with the `epoch` argument. Here however, we use the default. Again, if you want to run this notebook locally, make sure to adapt the folder name of the finetune run.

In [ ]:
# Change to the new finetuned model dir
finetuned_dir = "finetune_1110_180614"
finetuned_epoch = "020"
gpu_for_eval = 12

finetune_dir = Path(f"./runs/{finetuned_dir}")
eval_run(finetune_dir, period="test")

Now let's look at the test period results of the pre-trained base model and the finetuned model for the basin that we chose above.

In [ ]:
# load test results of the base run
# df_pretrained = pd.read_csv(run_dir / "test/model_epoch010/test_metrics.csv", dtype={'basin': str})
# df_pretrained = df_pretrained.set_index("basin")
    
# load test results of the finetuned model
df_finetuned = pd.read_csv(finetune_dir / f"test/model_epoch{finetuned_epoch}/test_metrics.csv", dtype={'basin': str})
df_finetuned = df_finetuned.set_index("basin")
    
# extract basin performance
# base_model_nse = df_pretrained.loc[df_pretrained.index == basin, "NSE"].values[0]
# finetune_nse = df_finetuned.loc[df_finetuned.index == basin, "NSE"].values[0]

# print(f"Basin {basin} base model performance: {base_model_nse:.3f}")
# print(f"Performance after finetuning: {finetune_nse:.3f}")
print(df_finetuned)

So we see roughly the same performance increase in the test period (slightly higher), which is great. However, note that a) our base model was not optimally trained (we stopped quite early) but also b) the finetuning settings were chosen rather randomly. From our experience so far, you can almost always get performance increases for individual basins with finetuning, but it is difficult to find settings that are universally applicable. However, this tutorial was just a showcase of how easy it actually is to finetune models with the NeuralHydrology library. Now it is up to you to experiment with it.

In [ ]:
def eval_results(period: str, epoch: str = "010") -> pd.DataFrame:
    df = pd.read_csv(Path(f"./runs/{finetuned_dir}") / f"{period}" / f"model_epoch{finetuned_epoch}" / f"{period}_metrics.csv", dtype={'basin': str})
    df = df.set_index('basin')

    # Compute the median NSE from all basins, where discharge observations are available for that period
    print(f"Median NSE of the {period} period {df['NSE'].median():.3f}")
    print(f"Mean NSE of the {period} period {df['NSE'].mean():.3f}")
    print(f"Median KGE of the {period} period {df['KGE'].median():.3f}")
    print(f"Mean KGE of the {period} period {df['KGE'].mean():.3f}")
    
    return df

In [ ]:
eval_run(finetune_dir, period="train", epoch=int(finetuned_epoch), gpu=gpu_for_eval)
df_train = eval_results("train", epoch=finetuned_epoch)

In [ ]:
eval_run(finetune_dir, period="validation", epoch=int(finetuned_epoch), gpu=gpu_for_eval)
df_val = eval_results("validation", epoch=finetuned_epoch)

In [ ]:
eval_run(finetune_dir, period="test", epoch=int(finetuned_epoch), gpu=gpu_for_eval)
df_test = eval_results("test", epoch=finetuned_epoch)

In [38]:
df_train = df_train.rename(columns={"NSE": "NSE_train", "KGE": "KGE_train"})
df_val = df_val.rename(columns={"NSE": "NSE_val", "KGE": "KGE_val"})
df_test = df_test.rename(columns={"NSE": "NSE_test", "KGE": "KGE_test"})

df_tmp = pd.merge(df_train, df_val, left_index=True, right_index=True)
df = pd.merge(df_tmp, df_test, left_index=True, right_index=True)

df.to_csv(finetune_dir / "eval.csv")